In [1]:
#利用tensorboard绘制网络结构
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

#每个批次的大小(表示每一次放入神经网络的图片数)
batch_size = 100
#计算一共有多少的批次
n_batch = mnist.train.num_examples//batch_size

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)#标准差
        tf.summary.scalar('max',tf.reduce_max(var))#最大值
        tf.summary.scalar('min',tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram',var)#直方图
        
#定义命名空间
with tf.name_scope('input'):
    #定义两个占位符
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')

#定义层级命名
with tf.name_scope('layer'):
    # W1 = tf.Variable(tf.random_normal([50,10]))
    with tf.name_scope('weight'):
        W = tf.Variable(tf.zeros([784,10]),name='w')
        variable_summaries(W)
    with tf.name_scope('bia'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b)
    with tf.name_scope('prediction'):
        prediction = tf.matmul(x,W)+b
        variable_summaries(prediction)

#二次代价
# loss = tf.reduce_mean(tf.square(y-prediction))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
    
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
# train_step = tf.train.AdamOptimizer(1e-2).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope('cor_acc'):
    with tf.name_scope('correct_prediction'):
        #结果存放在布尔类型列表中（argmax返回一维张量中最大的值所在的位置）
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        #把布尔类型转化成成0，1模式
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)
        
#合并所有的summary
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(10):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
            
        #writer.add_summary(summary,epoch)
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('iter '+str(epoch)+'  ,testing accuracy: '+str(acc))
        
        
#利用tensorboard --logdir 路径(logs文件夹的路径)  获取地址查看graph

W0809 12:49:01.518611 140735731680128 deprecation.py:323] From <ipython-input-1-220381cc22cf>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0809 12:49:01.521338 140735731680128 deprecation.py:323] From /Users/kosaki/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0809 12:49:01.524883 140735731680128 deprecation.py:323] From /Users/kosaki/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a f

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0809 12:49:01.863347 140735731680128 deprecation.py:323] From /Users/kosaki/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0809 12:49:01.868014 140735731680128 deprecation.py:323] From /Users/kosaki/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0809 12:49:01.955422 140735731680128 deprecation.py:323] From /Users/kosaki/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W0809 12:49:03.153614 140735731680128 deprecation.py:323] From <ipython-input-1-220381cc22cf>:46: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



iter 0  ,testing accuracy: 0.9085
iter 1  ,testing accuracy: 0.9165
iter 2  ,testing accuracy: 0.9178
iter 3  ,testing accuracy: 0.92
iter 4  ,testing accuracy: 0.9216
iter 5  ,testing accuracy: 0.9222
iter 6  ,testing accuracy: 0.9219
iter 7  ,testing accuracy: 0.9238
iter 8  ,testing accuracy: 0.9225
iter 9  ,testing accuracy: 0.9217
